In [1]:
#Installations
!pip install pandas
!pip install tslearn
!pip install prophet
!pip install pmdarima

In [2]:
from prophet import Prophet 

Importing plotly failed. Interactive plots will not work.


In [2]:
## Outside libraries
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
import pandas as pd
import os
import pandas as pd 
import numpy as np 
import itertools
from sklearn.metrics import mean_squared_error as mse

In [3]:
#Customer Functions
import config
import Forecast_01
import trainandtest
import transform
import AlgoArima
import HoltWinters

In [4]:
category = 'J'
config.DICT_JEW_PATH[category]

'Sales_Data.xlsx'

In [9]:
df = pd.read_excel('Sales_Data.xlsx')
df
unique=set(df.loc[:,level])

In [13]:
#Customer Functions
import config
import Forecast_01
category = 'J'
n=3
level="ID"
frequency="M"
n=1
salesFromPath=True
flagsFromPath=True
transformation=""
confidencelevel=0.95
sales, flags = Forecast_01.dataExtraction(category, level,salesFromPath,
                                                    flagsFromPath)
unique=set(sales["ID"])
algotable = pd.DataFrame()
forecast = pd.Series()

def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true.sum() - y_pred.sum()) / y_true.sum())) * 100



#Looping over every individual item in col "ID"
for unit in unique:
    #Subsetting sales and flags data for each unique item in "ID" column of
    #salesdata df for training and forecasting 
    print("Starting forecast for CATPB: ",unit)
    salesdata = pd.Series(sales.loc[sales["ID"]==unit,"SALES"])
    startdate = salesdata.index[0]
    #startdate = salesdata[np.min(salesdata):].index[0]
    salesdata = salesdata[startdate:]
    flagsdata = flags[startdate:]  

    #Grouping data to a weekly level for training/forecasting
    if frequency=="W":
        salesdata = salesdata.resample('W').sum()
        flagsdata = flagsdata.drop('WEEKEND_FLAG',axis=1,errors='ignore')
        flagsdata = flagsdata.resample('W').mean()

    #Grouping data to a monthly level for training/forecasting
    if frequency=="M":
        salesdata = salesdata.resample('M').sum()
        flagsdata = flagsdata.drop(["Start of Month Flag","MONTHEND_FLAG",
                                    "End of Month Flag"],axis=1,errors='ignore')
        flagsdata = flagsdata.resample('M').mean()        

    flagsdf = flagsdata.iloc[len(salesdata):len(salesdata)+n]   
    flagsdata = flagsdata.iloc[:len(salesdata)]        

    ######### TRAIN-TEST SPLIT #########
    trainsales, testsales = trainandtest.traintestsplit(salesdata)
    trainflags, testflags = trainandtest.traintestsplit(flagsdata)

    ############################# MODEL TRAINING ############################# 
    #creating dataframe for storing mse for models 
    errortable = pd.DataFrame()
    
        ########## ARIMA ##########

    #Transforming data to stationarize it 
    transformeddata = transform.transform(trainsales,confidencelevel,
                                          transformation)
    #Passing scaled data to the arimax model 
    scaledforecast = AlgoArima.arimax(transformeddata[0],len(testsales))
    #Inverse scaling the scaled forecasts 
    arimaforecasts = transform.inverse(scaledforecast,transformeddata)
    error = mse(arimaforecasts ,testsales)
    error2 = MAPE(testsales,arimaforecasts)
    #Appending mse and MAPE to error table 
    errortable = errortable.append({'Algorithm' : 'ARIMA', 'mse' : error, 
                                    "MAPE":error2},ignore_index=True)            
    del scaledforecast, arimaforecasts
    
    trainflags1 = trainflags.iloc[-len(transformeddata[0]):,:]
    transformeddata[0].index = trainflags1.index
    scaledforecast = AlgoArima.arimax(transformeddata[0],len(testsales),
                                 trainFlags=trainflags1,forecastFlags=testflags)
    arimaxforecasts = transform.inverse(scaledforecast,transformeddata)
    error = mse(arimaxforecasts ,testsales)
    error2 = MAPE(testsales,arimaxforecasts)
    errortable = errortable.append({'Algorithm' : 'ARIMAX', 'mse' : error,
                                    "MAPE":error2},ignore_index=True)
    del transformeddata, scaledforecast, arimaxforecasts, trainflags1
    
    holtMSE,algo = HoltWinters.HOLTWINTER(trainsales,testsales,category,
                                                frequency)
    if algo == "AANd":
        #AANd = Additive trend , Additive seasonality , No Damp
        errortable = errortable.append({'Algorithm' : 'HOLT-WINTERS-AANd', 
                                        'mse' : holtMSE},ignore_index=True)                                     

    elif algo == "AMNd":    
        #AANd = Additive trend , Multiplicative seasonality , No Damp
        errortable = errortable.append({'Algorithm' : 'HOLT-WINTERS-AMNd', 
                                        'mse' : holtMSE},ignore_index=True)                                 
    elif algo == "AAD":
        #AAD = Additive trend , Additive seasonality , With Damp
        errortable = errortable.append({'Algorithm' : 'HOLT-WINTERS-AAD', 
                                        'mse' : holtMSE},ignore_index=True)                                          
    elif algo == "AMD":
        #AMD = Additive trend , Multiplicative seasonality , No Damp
        errortable = errortable.append({'Algorithm' : 'HOLT-WINTERS-AMD', 
                                        'mse' : holtMSE},ignore_index=True)
    alg,MSE,mape = AlgoArima.allcombTSLM(trainsales,testsales,frequency,n,trainflags,testflags)
    errortable = errortable.append({'Algorithm' : alg, 'mse' : MSE,  
                                        'MAPE' : mape},ignore_index=True)      
        
          
    errortable.dropna(inplace=True)
    algo = errortable.loc[(errortable['MAPE'].idxmin())]
    print("Best algorithm for forecasting ",unit," is ",algo.Algorithm, ", with error: ",algo.MAPE)
    algotable = algotable.append({"ID":unit,"algorithm":algo.Algorithm,"MAPE":algo.MAPE},ignore_index=True)




Started
Extraction_started
Flags data shape: (2557, 24)
Sales data shape: (12798, 3)
Min_data 2014-04-01 00:00:00
Max_data 2020-01-31 00:00:00
Starting forecast for CATPB:  South 2
Best algorithm for forecasting  South 2  is  ARIMA , with error:  1.5955954927809195
Starting forecast for CATPB:  North
Best algorithm for forecasting  North  is  ARIMAX , with error:  3.2721520918013267
Starting forecast for CATPB:  East
Best algorithm for forecasting  East  is  (AM)TSLMwFlags(Trend) , with error:  4.116091403045879
Starting forecast for CATPB:  South 1
Best algorithm for forecasting  South 1  is  ARIMA , with error:  0.009749618989201717
Starting forecast for CATPB:  West
Best algorithm for forecasting  West  is  ARIMA , with error:  3.4908946001276835
Starting forecast for CATPB:  Overall
Best algorithm for forecasting  Overall  is  ARIMA , with error:  4.321273458384978


In [15]:
algotable.to_excel("Trial_Data.xlsx")

In [7]:
#Defining MAPE metric 
def TSLM(data,f="D",n=1,remove_trend=False,remove_seasonality=False,seasonality="additive",
         trainFlags=None,forecastFlags=None,only_error=False,testdata=None):
    """
    f- (seasonality): daily-"D" , weekly-"W", monthly-"M" 
    data should be a pandas series with sales data, flags a pandas dataframe with 
    exog var split into train and test 
    set remove_trend/remove_seasonality to True to run model after removing trend 
    and seasonality 
    n - no.of periods into future to forecast 
    """
    
    if f=="D":
        frequency = 365
    elif f=="W":
        frequency = 52
    elif f=="M":
        frequency = 12
    #Decomposing data into trend,seasonality and residual components
    result = seasonal_decompose(data,model=seasonality, freq=frequency,extrapolate_trend=True)
    
    # Removes trend component from data
    if remove_trend==True:
        if seasonality=="additive":
            data = data - result.trend 
        if seasonality=="multiplicative":
            data = data/result.trend 
            
    #Removes seasonality component from data
    if remove_seasonality==True:
        if seasonality=="additive":
            data = data - result.seasonal
        if seasonality=="multiplicative":
            data = data/result.seasonal
        
    #Calculates optimal value of p(hyperparameter) for fitting the AR model 
    pacfvalues=pacf(data,nlags=(data.shape[0] // 2)-1)
    CI = 1.96/(np.sqrt(len(data)))
    paramp=np.min(np.nonzero(abs(pacfvalues)<CI))-1

    #Fits AR model and returns predicted future sales 
    forecast = arimax(data,n,pval=paramp,qval=0,trainFlags=trainFlags,forecastFlags=forecastFlags)
    
    #Creates line of best fit(on trend component) and extrapolates it for future trend
    #Adds appropriate Datatime index to future trend values and adds to forecasted 
    #residual values 
    if remove_trend==True:
        trend = np.polyfit(np.array(range(1,len(data)+1)),result.trend.values,1)
        extrapolateTrend = np.poly1d(trend)
        futureTrend = pd.Series(extrapolateTrend(range(len(data),len(data)+1+n)))
        futureTrend.index = pd.date_range(start=data.index[-1],periods=n+1,freq=f)
        if seasonality=="additive":
            forecast = forecast + futureTrend[1:] 
        if seasonality=="multiplicative":
            forecast = forecast * futureTrend[1:] 
    
    #Adds back seasonality component 
    if remove_seasonality==True:
        if frequency>n:
            futureSeasonal = pd.Series(result.seasonal.values[-frequency:-frequency+n+1])
        elif frequency<=n:
            futureSeasonalCycle = pd.Series(result.seasonal.values[-frequency:])
            count = int(n/frequency)
            futureSeasonal = futureSeasonalCycle
            for i in range(count):
                futureSeasonal = futureSeasonal.append(futureSeasonalCycle)
            futureSeasonal = futureSeasonal[:n+1]
        futureSeasonal.index = pd.date_range(start=data.index[-1],periods=n+1,freq=f)
        if seasonality=="additive":
            forecast = forecast + futureSeasonal[1:]
        if seasonality=="multiplicative":
            forecast = forecast * futureSeasonal[1:] 
        
        
    #forecast = np.round(forecast)
    if only_error==False:
        return forecast 
    
    else:
        error = mean_squared_error(forecast,testdata)
        return error
        
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def allcombTSLM(trainsales,testsales,frequency,n,trainflags,testflags):
    print("Started")
    
    #creating dataframe for storing MSE for models 
    errortable = pd.DataFrame()
    
    ############ TSLM WITHOUT FLAGS (MODEL - ADDITIVE) ############
    # AM - ADDITIVE MOEDL 
    ### RESIDUAL + TREND + SEASONALITY ###
    try:
        tslmForecast = TSLM(trainsales,f=frequency,n=len(testsales))
        error = MSE(tslmForecast,testsales)
        error2 = MAPE(testsales,tslmForecast)
        errortable = errortable.append({'Algorithm' : '(AM)TSLM', 'MSE' : error,
                                       "MAPE":error2},
                                       ignore_index=True)
    except:
        errortable = errortable.append({'Algorithm':'(AM)TSLM','MSE':np.nan},
                                       ignore_index=True)  
        
    ### RESIDUAL + SEASONALITY ###
    try:
        tslmForecast = TSLM(trainsales,f=frequency,n=len(testsales),
                                 remove_trend=True)
        error = MSE(tslmForecast,testsales)
        error2 = MAPE(testsales,tslmForecast)
        errortable = errortable.append({'Algorithm' : '(AM)TSLM(Trend)', 
                                        'MSE' : error,"MAPE":error2},
                                       ignore_index=True)            
    except:
        errortable = errortable.append({'Algorithm' : '(AM)TSLM(Trend)', 
                                        'MSE':np.nan},ignore_index=True)             
    
    ### RESIDUAL + TREND ###
    try:
        tslmForecast = TSLM(trainsales,f=frequency,n=len(testsales),
                                 remove_seasonality=True,)
        error = MSE(tslmForecast,testsales)
        error2 = MAPE(testsales,tslmForecast)
        errortable = errortable.append({'Algorithm' : '(AM)TSLM(Seas)', 
                                        'MSE' : error,"MAPE":error2},
                                       ignore_index=True)            
    except:
        errortable = errortable.append({'Algorithm' : '(AM)TSLM(Seas)', 
                                        'MSE':np.nan},ignore_index=True)               
    
    
    ### RESIDUAL ###
    try:
        tslmForecast = TSLM(trainsales,f=frequency,n=len(testsales),
                             remove_trend=True,remove_seasonality=True,)
        error = MSE(tslmForecast,testsales)
        error2 = MAPE(testsales,tslmForecast)
        errortable = errortable.append({'Algorithm' : '(AM)TSLM(Trend+Seas)', 
                                        'MSE' : error,"MAPE":error2},
                                       ignore_index=True)            
    except:
        errortable = errortable.append({'Algorithm' : '(AM)TSLM(Trend+Seas)', 
                                        'MSE':np.nan},ignore_index=True)         
    
    ############ TSLM WITH FLAGS (MODEL - ADDITIVE) ############
    ### RESIDUAL + TREND + SEASONALITY ###
    try:
        tslmForecast = TSLM(trainsales,f=frequency,n=len(testsales),
                                 trainFlags=trainflags,forecastFlags=testflags)
        error = MSE(tslmForecast,testsales)
        error2 = MAPE(testsales,tslmForecast)
        errortable = errortable.append({'Algorithm':'(AM)TSLMwFlags','MSE':error,
                                        "MAPE":error2}, ignore_index=True)
    except:
        errortable = errortable.append({'Algorithm' : '(AM)TSLMwFlags', 
                                        'MSE' : np.nan},ignore_index=True)            
    
    ### RESIDUAL + SEASONALITY ###
    try:
        tslmForecast = TSLM(trainsales,f=frequency,n=len(testsales),
                                 remove_trend=True,trainFlags=trainflags,
                                 forecastFlags=testflags)
        error = MSE(tslmForecast,testsales)
        error2 = MAPE(testsales,tslmForecast)
        errortable = errortable.append({'Algorithm' : '(AM)TSLMwFlags(Trend)', 
                                        'MSE' : error,"MAPE":error2},
                                       ignore_index=True)
    except:
        errortable = errortable.append({'Algorithm' : '(AM)TSLMwFlags(Trend)', 
                                        'MSE':np.nan},ignore_index=True)
    
    ### RESIDUAL + TREND ###
    try:
        tslmForecast = TSLM(trainsales,f=frequency,n=len(testsales),
                                 remove_seasonality=True,trainFlags=trainflags,
                                 forecastFlags=testflags)
        error = MSE(tslmForecast,testsales)
        error2 = MAPE(testsales,tslmForecast)
        errortable = errortable.append({'Algorithm' : '(AM)TSLMwFlags(Seas)',
                                        'MSE' : error,"MAPE":error2},
                                       ignore_index=True)
    except:
        errortable = errortable.append({'Algorithm' : '(AM)TSLMwFlags(Seas)', 
                                        'MSE':np.nan},ignore_index=True)            
    
    ### RESIDUAL ###
    try:
        tslmForecast = TSLM(trainsales,f=frequency,n=len(testsales),
                                 remove_trend=True,remove_seasonality=True,
                                 trainFlags=trainflags,forecastFlags=testflags)
        error = MSE(tslmForecast,testsales)
        error2 = MAPE(testsales,tslmForecast)
        errortable = errortable.append({'Algorithm':'(AM)TSLMwFlags(Trend+Seas)',
                                        'MSE' : error,"MAPE":error2},
                                       ignore_index=True)
    except:
        errortable = errortable.append({'Algorithm':'(AM)TSLMwFlags(Trend+Seas)', 
                                        'MSE':np.nan},ignore_index=True)             

    
    if not any(trainsales<1):                   
        # MM - MULTIPLICATIVE MODEL                
        ############ TSLM WITHOUT FLAGS (MODEL - MULTIPLICATIVE) ############  
        ### RESIDUAL * SEASONALITY ###
        try:
            tslmForecast = TSLM(trainsales,f=frequency,n=len(testsales),
                                     seasonality="multiplicative",remove_trend=True)
            error = MSE(tslmForecast,testsales)
            error2 = MAPE(testsales,tslmForecast)
            errortable = errortable.append({'Algorithm' : '(MM)TSLM(Trend)', 
                                            'MSE' : error,"MAPE":error2},
                                           ignore_index=True)            
        except:
            errortable = errortable.append({'Algorithm' : 'TSLM(Trend)', 
                                            'MSE':np.nan},ignore_index=True)             
        
        ### RESIDUAL * TREND ###
        try:
            tslmForecast = TSLM(trainsales,f=frequency,remove_seasonality=True,
                                     seasonality="multiplicative",n=len(testsales))
            error = MSE(tslmForecast,testsales)
            error2 = MAPE(testsales,tslmForecast)
            errortable = errortable.append({'Algorithm' : '(MM)TSLM(Seas)', 
                                            'MSE':error,"MAPE":error2},
                                           ignore_index=True)            
        except:
            errortable = errortable.append({'Algorithm' : '(MM)TSLM(Seas)', 
                                            'MSE':np.nan},ignore_index=True)               
        
        
        ### RESIDUAL ###
        try:
            tslmForecast = TSLM(trainsales,f=frequency,n=len(testsales),
                                     remove_trend=True,remove_seasonality=True,
                                     seasonality="multiplicative")
            error = MSE(tslmForecast,testsales)
            error2 = MAPE(testsales,tslmForecast)
            errortable = errortable.append({'Algorithm' : '(MM)TSLM(Trend+Seas)', 
                                            'MSE' : error,"MAPE":error2},
                                           ignore_index=True)            
        except:
            errortable = errortable.append({'Algorithm' : '(MM)TSLM(Trend+Seas)', 
                                            'MSE':np.nan},ignore_index=True)
        
        ############ TSLM WITH FLAGS (MODEL - MULTIPLICATIVE) ###########       
        ### RESIDUAL * SEASONALITY ###
        try:
            tslmForecast = TSLM(trainsales,f=frequency,remove_trend=True,
                                     trainFlags=trainflags,seasonality="multiplicative",
                                     n=len(testsales),forecastFlags=testflags)
            error = MSE(tslmForecast,testsales)
            error2 = MAPE(testsales,tslmForecast)
            errortable = errortable.append({'Algorithm' : '(MM)TSLMwFlags(Trend)', 
                                            'MSE' : error,"MAPE":error2},
                                           ignore_index=True)
        except:
            errortable = errortable.append({'Algorithm' : '(MM)TSLMwFlags(Trend)', 
                                            'MSE' : np.nan},ignore_index=True)
        
        ### RESIDUAL * TREND ###
        try:
            tslmForecast = TSLM(trainsales,f=frequency,remove_seasonality=True,
                                     seasonality="multiplicative",n=len(testsales),
                                     trainFlags=trainflags,forecastFlags=testflags)
            error = MSE(tslmForecast,testsales)
            error2 = MAPE(testsales,tslmForecast)
            errortable = errortable.append({'Algorithm' : '(MM)TSLMwFlags(Seas)', 
                                            'MSE' : error,"MAPE":error2},
                                           ignore_index=True)
        except:
            errortable = errortable.append({'Algorithm' : '(MM)TSLMwFlags(Seas)', 
                                            'MSE' : np.nan},ignore_index=True)            
        
        ### RESIDUAL ###
        try:
            tslmForecast = TSLM(trainsales,f=frequency,n=len(testsales),
                                     remove_trend=True,trainFlags=trainflags,
                                     remove_seasonality=True,forecastFlags=testflags,
                                     seasonality="multiplicative")
            error = MSE(tslmForecast,testsales)
            error2 = MAPE(testsales,tslmForecast)
            errortable = errortable.append({'Algorithm' : '(MM)TSLMwFlags(Trend+Seas)', 
                                            'MSE' : error,"MAPE":error2},
                                           ignore_index=True)
        except:
            errortable = errortable.append({'Algorithm' : '(MM)TSLMwFlags(Trend+Seas)', 
                                            'MSE' : np.nan},ignore_index=True)
        
        #Extracting algorithm name and MSE,MAPE of best algorithm 
    print(errortable)
    row = errortable.loc[errortable['MSE'].idxmin()]
    algorithm    = row["Algorithm"]
    mse = row["MSE"]
    mape = row["MAPE"]
    
    return algorithm,mse,mape
         
          




In [11]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
import numpy as np 
import pandas as pd
from sklearn.metrics import mean_squared_error as MSE


tslmForecast = AlgoArima.TSLM(trainsales,f=frequency,n=len(testsales))
AlgoArima.allcombTSLM(trainsales,testsales,frequency,n,trainflags,testflags)

('(MM)TSLMwFlags(Trend)', 1740425427448114.8, 7.021818825316408)

In [10]:
tslmForecast

2018-05-31    4.581502e+08
2018-06-30    4.404736e+08
2018-07-31    4.256085e+08
2018-08-31    4.131078e+08
2018-09-30    4.025953e+08
2018-10-31    3.937549e+08
2018-11-30    3.863205e+08
2018-12-31    3.800687e+08
2019-01-31    3.748111e+08
2019-02-28    3.703898e+08
2019-03-31    3.666718e+08
2019-04-30    3.635451e+08
2019-05-31    3.609157e+08
2019-06-30    3.587045e+08
2019-07-31    3.568450e+08
2019-08-31    3.552813e+08
2019-09-30    3.539662e+08
2019-10-31    3.528604e+08
2019-11-30    3.519304e+08
2019-12-31    3.511484e+08
2020-01-31    3.504907e+08
Freq: M, dtype: float64